In [6]:

# bundle = {"application/vnd.vega.v5+json": schema}
# print("", end=None)
# IPython.display.display(bundle, raw=True)





In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import  vega, plots
from pathlib import Path
import os
import json 

from IPython.display import display

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
import json
import random
from itertools import chain
from pathlib import Path

import networkx as nx
import numpy as np
import pandas as pd
import pytest

import pyciemss
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import plots, vega



In [4]:
# #'timepoint_id', 'sample_id'
# sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
# sine_distribution = sine_distribution.rename(
#             columns={
#                 'Unnamed: 0.1': "timepoint_id",
#                 'Unnamed: 0': "100",
#             }
#         )
# sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
#         var_name="sample_id", 
#         value_name="same_stat_state")
name = "all_functions_kmeans"
timepoint_list = []
sine_list = []
sample_list = []


def save_result(data, name, ref_ext):
    """Save new reference files"""
    # _output_root.mkdir(parents=True, exist_ok=True)
    _output_root = 'output'
    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)
        
def return_sine_function_3(function_name):
        if function_name == "initial_function":
                a, b, c, d = 0, 0, 0, np.random.randn()
                a2, b2, c2, d2 = 0, 0, 0, 0
        if function_name == "sine_function":
                a, b, c, d = 4*np.random.randn(), .25*np.random.randn(), np.pi*np.random.randn(), np.random.randn()
                a2, b2, c2, d2 = 2, .25, np.pi, 0
        if function_name == "sine_function_2":
                a, b, c, d = 4, .25*np.random.randn(), 2*np.pi*np.random.randn(), np.random.randn()
                a2, b2, c2, d2 = 4, .5, 2*np.pi, 0
        if function_name == "sine_function_3":
                a, b, c, d = 2, .25*np.random.randn(), np.pi*np.random.randn()/2, np.random.randn()
                a2, b2, c2, d2 = 2, .25, np.pi/2, 0
        def sine_function(x):
                return np.power(x, .25) + a*np.sin(b*x + c) + d
        def sine_function_default(x):
                return np.power(x, .25) + a2*np.sin(b2*x + c2) + d2
        function_parameters = {'a': np.round(a, 4), 'b': np.round(b, 4), 'c': np.round(c, 4), 'd':np.round(d, 4), 'a mean': a2, 'b mean': b2, 'c mean': c2, 'd mean':d2}
        return sine_function, sine_function_default, function_parameters

function_dataframe = pd.DataFrame(columns=['functions', 'sample_id', 'a', 'b', 'c', 'd', 'a_mean', 'b_mean', 'c_mean', 'c_mean'])
function_list = []
j = 0
all_functions ={}
for funct_name in ["initial_function", "sine_function", "sine_function_2", "sine_function_3"]:
    
    for _ in range(5):
        j+=1
        timepoint_list.extend([x for x in range(100)])
        sample_list.extend([j]*100)
        random_function,  sine_function_default, function_parameters = return_sine_function_3(funct_name)

        function_parameters['functions'] = funct_name
        function_parameters['sample_id'] = j
        function_list.append(function_parameters)
        sine_list.extend([random_function(x) for x in range(100)])
    all_functions[funct_name] = sine_function_default 

function_dataframe = pd.DataFrame(function_list)

sine_distribution= pd.DataFrame({'timepoint_id' : timepoint_list,
                   'sample_id' : sample_list,
                   'Example': sine_list})


traces_list = []   
 
#select_by "mean", "var", "granger"
examplary_line = pyciemss.visuals.trajectories.select_traces(sine_distribution, kmean=True)
best_samples_cluster = examplary_line[['sample_id', 'select_by', 'cluster']].drop_duplicates()
best_samples_df_list = []
for cluster in list(np.unique(examplary_line['cluster'])):    
        current_cluster_df = examplary_line[examplary_line['cluster'] == cluster]
        current_cluster_df = current_cluster_df.reset_index()
        examplary_lines_pivot = current_cluster_df[['examplary_line', 'mean_trajectory', 'timepoint', 'select_by']].pivot_table(
        values="examplary_line", index = ["timepoint", 'mean_trajectory'], columns="select_by"
        )
        examplary_lines_pivot = examplary_lines_pivot.reset_index()
        examplary_lines_pivot = examplary_lines_pivot.rename(columns = {'mean_trajectory': 'Baseline_Mean'})
        # for function_name, function in all_functions.items():
        #         examplary_lines_pivot["Baseline_" + function_name] = [function(x) for x in range(100)]
        schema = plots.trajectories(sine_distribution[['timepoint_id', 'sample_id', 'Example']], traces = examplary_lines_pivot)
        path = "{}_{}.json".format(name, cluster)
        image = plots.ipy_display(schema, format="PNG").data
        save_result(image, path, "png")
        plots.ipy_display(schema, format="interactive")

all_clusters = best_samples_cluster.merge(function_dataframe, how="inner", on="sample_id")
all_clusters.to_csv("output/{}.csv".format(name))
# table with sine variables of starting function and the formula of the best select_by lines and metrics1

Init 1
185.485 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


60.420 --> 60.410 --> 60.410 --> 
Init 2


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


157.906 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


60.004 --> 60.004 --> 

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-6.0350 , p=1.0000  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=-78.7180, p=1.0000  , df=10
likelihood ratio test: chi2=-186.8939, p=1.0000  , df=10
parameter F test:         F=0.2006  , p=0.9843  , df_denom=69, df_num=7

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-0.1411 , p=1.0000  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=-1.8399 , p=1.0000  , df=10
likelihood ratio test: chi2=-1.8590 , p=1.0000  , df=10
parameter F test:         F=1.5661  , p=0.1603  , df_denom=69, df_num=7

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-6.4628 , p=1.0000  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=-84.2973, p=1.0000  , df=10
likelihood ratio test: chi2=-248.2988, p=1.0000  , df=10
parameter F test:         F=0.1015  , p=0.9981  , df_denom=69, df_num=7

Granger Causality
number of lags (no zero) 10
ssr based F test

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=9.1592  , p=0.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=104.3453, p=0.0000  , df=10
likelihood ratio test: chi2=69.2844 , p=0.0000  , df=10
parameter F test:         F=949638.8271, p=0.0000  , df_denom=79, df_num=6


/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '


In [ ]:
def distributions():
    model_1_path = (
            "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration"
            "/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
    )
    start_time = 0.0
    end_time = 100.0
    logging_step_size = 1
    num_samples = 30
    sample = pyciemss.sample(
        model_1_path,
        end_time,
        logging_step_size,
        num_samples,
        start_time=start_time,
        solver_method="euler",
    )["unprocessed_result"]

    for e in sample.values():
        if len(e.shape) > 1:
            num_timepoints = e.shape[1]

    return convert_to_output_format(
        sample,
        timepoints=np.linspace(start_time, end_time, num_timepoints),
        time_unit="notional",
    )

distributions = distributions()
print(distributions.head())
print(distributions.columns)
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state',
       'exposed_observable_state', 'hospitalized_observable_state',
       'dead_observable_state']
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state']

distributions = distributions[all_columns]
distributions

   timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
0             0          0                  0.39528                0.574258   
1             1          0                  0.39528                0.574258   
2             2          0                  0.39528                0.574258   
3             3          0                  0.39528                0.574258   
4             4          0                  0.39528                0.574258   

   persistent_gamma_param  persistent_hosp_param  persistent_death_hosp_param  \
0                0.454567               0.117964                     0.083155   
1                0.454567               0.117964                     0.083155   
2                0.454567               0.117964                     0.083155   
3                0.454567               0.117964                     0.083155   
4                0.454567               0.117964                     0.083155   

   persistent_I0_param   D_state    E_

,timepoint_id,sample_id,infected_observable_state
0,0,0,10.749370
1,1,0,13.560308
2,2,0,14.712402
3,3,0,15.458530
4,4,0,16.119194
...,...,...,...
2965,94,29,14.551717
2966,95,29,13.349472
2967,96,29,12.240627
2968,97,29,11.221556


In [ ]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")

TypeError: select_traces() got an unexpected keyword argument 'select_by'

In [ ]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")



UnboundLocalError: local variable 'all_trajectories_df' referenced before assignment

In [ ]:


traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(distributions, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, format="interactive")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=15.6973 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=205.4503, p=0.0000  , df=10
likelihood ratio test: chi2=106.4862, p=0.0000  , df=10
parameter F test:         F=15.6840 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=0.1657  , p=0.9980  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=2.1684  , p=0.9949  , df=10
likelihood ratio test: chi2=2.1424  , p=0.9951  , df=10
parameter F test:         F=0.1138  , p=0.9986  , df_denom=68, df_num=8

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=4.9416  , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=64.6771 , p=0.0000  , df=10
likelihood ratio test: chi2=48.6133 , p=0.0000  , df=10
parameter F test:         F=4.9250  , p=0.0000  , df_denom=68, df_num=9

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=6.2662  , p=

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [ ]:

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(distributions, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, format="interactive")

Init 1
2209424658818.259 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


506050553067.124 --> 506050553066.406 --> 506050553066.406 --> 
Init 2


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


2505050446895.445 --> 

...

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based chi2 test:   chi2=162.0532, p=0.0000  , df=10
likelihood ratio test: chi2=92.2955 , p=0.0000  , df=10
parameter F test:         F=15.2032 , p=0.0000  , df_denom=68, df_num=8

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=578.9103, p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=7576.9137, p=0.0000  , df=10
likelihood ratio test: chi2=396.5753, p=0.0000  , df=10
parameter F test:         F=578.9106, p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=681.2702, p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=8916.6248, p=0.0000  , df=10
likelihood ratio test: chi2=410.9102, p=0.0000  , df=10
parameter F test:         F=681.2829, p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=783.0103, p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=10248.2231, p=0.0000  , df=10
likelihood ratio test: chi

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 8
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=202.1290, p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=2645.5120, p=0.0000  , df=10
likelihood ratio test: chi2=304.8314, p=0.0000  , df=10
parameter F test:         F=202.1092, p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=204.5599, p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=2677.3279, p=0.0000  , df=10
likelihood ratio test: chi2=305.8609, p=0.0000  , df=10
parameter F test:         F=204.5599, p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=2.3503  , p=0.0189  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=30.7610 , p=0.0006  , df=10
likelihood ratio test: chi2=26.4207 , p=0.0032  , df=10
parameter F test:         F=2.7091  , p=0.0119  , df_denom=68, df_num=8

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=517.3829, p=0.0000  , df_denom=68, df_num=10
ssr based c

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [ ]:
%matplotlib inline

_output_root = "output_images/"

def save_result(data, name, ref_ext):
    """Save new reference files"""
    # _output_root.mkdir(parents=True, exist_ok=True)

    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)
traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(distributions, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)

new_df = pd.concat(traces_list, axis = 1)


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=57.9980 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=759.0921, p=0.0000  , df=10
likelihood ratio test: chi2=200.6374, p=0.0000  , df=10
parameter F test:         F=57.8450 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=12.5078 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=163.7056, p=0.0000  , df=10
likelihood ratio test: chi2=92.8794 , p=0.0000  , df=10
parameter F test:         F=12.5113 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=5.3391  , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=69.8789 , p=0.0000  , df=10
likelihood ratio test: chi2=51.5761 , p=0.0000  , df=10
parameter F test:         F=5.6117  , p=0.0000  , df_denom=68, df_num=9

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=9.3416  , p

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [ ]:
mean_line_df 

trajectory,infected_observable_state_compare_line_no_cluster
timepoint,
0,15.404147
1,19.096250
2,21.363398
3,23.200246
4,25.035779
...,...
94,357288.143164
95,344345.879027
96,332529.691912


In [ ]:
combined_df = pd.concat([new_df, mean_line_df], axis =1)
combined_df

trajectory,infected_observable_state_mean_no_cluster,infected_observable_state_var_no_cluster,infected_observable_state_granger_no_cluster,infected_observable_state_compare_line_no_cluster
timepoint,,,,
0,18.839336,18.839336,2.180719e+01,15.404147
1,24.788004,24.788004,2.778604e+01,19.096250
2,29.499735,29.499735,3.301510e+01,21.363398
3,33.763748,33.763748,3.821029e+01,23.200246
4,38.003677,38.003677,4.375787e+01,25.035779
...,...,...,...,...
94,321817.250000,321817.250000,2.377996e+06,357288.143164
95,324220.718750,324220.718750,2.474322e+06,344345.879027
96,325817.937500,325817.937500,2.564101e+06,332529.691912


In [ ]:
schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "traj_infected_observable", "png")


FileNotFoundError: [Errno 2] No such file or directory: 'output_images/traj_infected_observable.png'

In [ ]:
schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "traj_infected_observable", "png")


FileNotFoundError: [Errno 2] No such file or directory: 'output_images/traj_infected_observable.png'

In [ ]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=19.6658 , p=0.0326  , df=10
likelihood ratio test: chi2=17.7865 , p=0.0587  , df=10
parameter F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=23.2664 , p=0.0098  , df=10
likelihood ratio test: chi2=20.6939 , p=0.0233  , df=10
parameter F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=7.2420  , p=0.7024  , df=10
likelihood ratio test: chi2=6.9653  , p=0.7287  , df=10
parameter F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.1772  , 

In [ ]:
# #'timepoint_id', 'sample_id'
# sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
# sine_distribution = sine_distribution.rename(
#             columns={
#                 'Unnamed: 0.1': "timepoint_id",
#                 'Unnamed: 0': "100",
#             }
#         )
# sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
#         var_name="sample_id", 
#         value_name="same_stat_state")
timepoint_list = []
sine_list = []
sample_list = []
def initial_function(x):
     return np.power(x, .25) + (2*np.random.random_sample()-1)

def sine_function(x):
        return np.power(x, .25) + np.sin(x) + (2*np.random.random_sample()-1)

def sine_function_2(x):
        return np.power(x, .25) + 2*np.sin(x) + (2*np.random.random_sample()-1)

for funct_x in [initial_function, sine_function, sine_function_2]:
    for i in range(3):
        timepoint_list.extend([x for x in range(100)])
        sample_list.extend([i]*100)
        sine_list.extend([funct_x(x) for x in range(100)])
    
sine_distribution= pd.DataFrame({'timepoint_id' : timepoint_list,
                   'sample_id' : sample_list,
                   'same_stat_state': sine_list})



traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")

NameError: name 'np' is not defined

In [ ]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sin/paper_sine_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "mean")[0]
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
# traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "var")[0]["same_stat_state"]
# traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "granger")[0]["same_stat_state"]
schema = plots.trajectories(sine_distribution, traces = traces)

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_sin", "png")

FileNotFoundError: [Errno 2] No such file or directory: 'output_images/paper_sin.png'

In [ ]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sin/paper_sine_old_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "mean")
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "var")["same_stat_state"]
traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = "granger")["same_stat_state"]
schema = plots.trajectories(sine_distribution, traces = traces)
print(traces_var)
plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_start", "png")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8988 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9865, p=1.0000  , df=10
likelihood ratio test: chi2=-792.3666, p=1.0000  , df=10
parameter F test:         F=9832.5499, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8986 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9844, p=1.0000  , df=10
likelihood ratio test: chi2=-779.4979, p=1.0000  , df=10
parameter F test:         F=11157.7647, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.1956 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-81.9749, p=1.0000  , df=10
likelihood ratio test: chi2=-217.5511, p=1.0000  , df=10
parameter F test:         F=5832969.5496, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8784 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.7535, p=1.0000  , df=10
likelihood ratio test: chi2=-531.0184, p=1.0000  , df=10
parameter F test:         F=180680.8853, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8631 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.5799, p=1.0000  , df=10
likelihood ratio test: chi2=-483.0319, p=1.0000  , df=10
parameter F test:         F=305790.2533, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8977 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9736, p=1.0000  , df=10
likelihood ratio test: chi2=-732.1803, p=1.0000  , df=10
parameter F test:         F=18989.1084, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8779 , p=1.0000  , df_denom=79, df_num=10
ssr 

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8872 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.8541, p=1.0000  , df=10
likelihood ratio test: chi2=-578.2452, p=1.0000  , df=10
parameter F test:         F=105861.9438, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-3.7004 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-42.1567, p=1.0000  , df=10
likelihood ratio test: chi2=-56.8690, p=1.0000  , df=10
parameter F test:         F=35295813.9609, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8970 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9661, p=1.0000  , df=10
likelihood ratio test: chi2=-709.6945, p=1.0000  , df=10
parameter F test:         F=24424.8915, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8985 , p=1.0000  , df_denom=79, df_num=10
ssr

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

trajectory  infected_observable_state_var  timepoint_id
timepoint                                              
0                               11.969685             0
1                               15.828713             1
2                               18.445007             2
3                               20.906946             3
4                               23.540363             4
...                                   ...           ...
94                          348138.093750            94
95                          325088.843750            95
96                          302895.062500            96
97                          281833.750000            97
98                          261997.953125            98

[99 rows x 2 columns]

